In [1]:
%pip install -e ..
%load_ext autoreload
%autoreload 2


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Obtaining file:///C:/Users/USER/Desktop/projects/Health%20Insurance%20Model
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for insurance (pyproject.toml): started
  Building editable for insurance (pyproject.toml): finished with status 'done'
  Created wheel for insurance: filename=insurance-0.1.0-0.editable-py3-none-any.whl size=1307 sha256=ed701ab722f164ffaf59fa88b60eef5363b8ba8898ec93e7d3eb834ca9a54efd
  Stored in directory: C:\Users\USER\AppData\Local\Temp\pip-ephem-wheel-cache-zy1ep36d\wheels\c7\86\56\d85e9c1741a69fba656

In [10]:
import pandas as pd, numpy as np, joblib, pathlib, datetime as dt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns

from insurance.data.load import load_raw, AGE_PATH
from insurance.features.engineering import prepare_data, label_encode_categoricals
from insurance.features.risk_score import add_normalized_risk_score
from insurance.features.preprocessing import make_preprocessor

### Dataset building for Young Segment - including new feature: Genetical_Risk

In [7]:
df = load_raw(AGE_PATH)
df.sample(1)

,Age,Gender,Region,Marital_status,Number Of Dependants,BMI_Category,Smoking_Status,Employment_Status,Income_Level,Income_Lakhs,Medical History,Insurance_Plan,Annual_Premium_Amount,Genetical_Risk
6429,24,Male,Northwest,Unmarried,0,Normal,No Smoking,Freelancer,25L - 40L,31,Diabetes,Bronze,4650,0


In [8]:
df = prepare_data(df)
df = add_normalized_risk_score(df)
df.sample(1)

,age,gender,region,marital_status,number_of_dependants,bmi_category,smoking_status,employment_status,income_level,income_lakhs,medical_history,insurance_plan,annual_premium_amount,genetical_risk,disease1,disease2,total_risk_score,normalized_risk_score
2149,20,Male,Northeast,Unmarried,0,Normal,Occasional,Freelancer,<10L,3,No Disease,Bronze,5395,1,no disease,none,0,0.0


In [11]:
df = label_encode_categoricals(df)
df.sample(1)

,age,gender,region,marital_status,number_of_dependants,bmi_category,smoking_status,employment_status,income_lakhs,medical_history,insurance_plan,annual_premium_amount,genetical_risk,disease1,disease2,total_risk_score,normalized_risk_score
3721,21,Male,Southwest,Unmarried,2,Normal,Occasional,Salaried,16,No Disease,0,4276,0,no disease,none,0,0.0
